In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import IPython.display as display
from functools import partial
import matplotlib.pyplot as plt
import glob
from PIL import Image, ImageDraw
from datetime import datetime

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("TensorFlow version: ", tf.__version__)

Num GPUs Available:  1
TensorFlow version:  2.14.0


In [4]:
def parse_tfrecord_fn(example_proto):
    # Define the feature description for parsing
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'steering': tf.io.FixedLenFeature([], tf.float32),
    }
    
    parsed_features = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.image.decode_jpeg(parsed_features['image'], channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.image.rgb_to_yuv(image)
    image = (image / 127.5) - 1.0
    
    return image, parsed_features['steering']


In [5]:
def load_dataset(tfrecord_files):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_files)
    parsed_dataset = raw_dataset.map(parse_tfrecord_fn)
    return parsed_dataset

In [6]:
# Create a sequential model
model = tf.keras.Sequential([
    # First convolutional layer, input shape specified
    layers.InputLayer((66, 200, 3)),
    layers.Conv2D(24, kernel_size=5, strides=2, padding='valid'),
    layers.BatchNormalization(),
    layers.ReLU(),

    # Second convolutional layer
    layers.Conv2D(36, kernel_size=5, strides=2, padding='valid'),
    layers.BatchNormalization(),
    layers.ReLU(),

    # Third convolutional layer
    layers.Conv2D(48, kernel_size=5, strides=2, padding='valid'),
    layers.BatchNormalization(),
    layers.ReLU(),

    # Fourth convolutional layer
    layers.Conv2D(64, kernel_size=3, strides=1, padding='valid'),
    layers.BatchNormalization(),
    layers.ReLU(),

    # Fifth convolutional layer
    layers.Conv2D(64, kernel_size=3, strides=1, padding='valid'),
    layers.BatchNormalization(),
    layers.ReLU(),

    # Flatten the output to feed into the dense layers
    layers.Flatten(),
    layers.Dropout(0.5),

    # First fully connected layer
    layers.Dense(1164),
    layers.BatchNormalization(),
    layers.ReLU(),

    # Second fully connected layer
    layers.Dense(100),
    layers.BatchNormalization(),
    layers.ReLU(),

    # Third fully connected layer
    layers.Dense(50),
    layers.BatchNormalization(),
    layers.ReLU(),

    # Fourth fully connected layer
    layers.Dense(10),
    layers.ReLU(),

    # Output layer
    layers.Dense(1)
])

# Summary of the model to see the structure and parameters
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 31, 98, 24)        1824      
                                                                 
 batch_normalization (Batch  (None, 31, 98, 24)        96        
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 31, 98, 24)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 47, 36)        21636     
                                                                 
 batch_normalization_1 (Bat  (None, 14, 47, 36)        144       
 chNormalization)                                                
                                                                 
 re_lu_1 (ReLU)              (None, 14, 47, 36)        0

In [7]:
tfrecord_files = list(glob.glob("/home/anaya/Develop/autonomous_driving_training/datasets/*.tfrecord"))
parsed_dataset = load_dataset(tfrecord_files)
shuffled_dataset = parsed_dataset.shuffle(2040)

# Determine split sizes
total_items = sum([1 for _ in shuffled_dataset.as_numpy_iterator()])
train_size = int(0.6 * total_items)
val_size = int(0.2 * total_items)
test_size = total_items - train_size - val_size


In [8]:
print("Total items: ", total_items)
print("Train size: ", train_size)
print("Validation size: ", val_size)
print("Test size: ", test_size)

Total items:  123138
Train size:  73882
Validation size:  24627
Test size:  24629


In [9]:
# Split the dataset
train_dataset = shuffled_dataset.take(train_size)
test_val_dataset = shuffled_dataset.skip(train_size)
val_dataset = test_val_dataset.take(val_size)
test_dataset = test_val_dataset.skip(val_size)

In [10]:
# Apply any additional preprocessing here (e.g., batching)
train_dataset = train_dataset.batch(64, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(64, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(64, drop_remainder=True).prefetch(tf.data.AUTOTUNE)


In [ ]:
# for data_row in train_dataset.take(1):
#     print(data_row[1][0])
#     image = Image.fromarray(((data_row[0][0].numpy() + 1.0) * 127.5).astype(np.uint8))
#     display.display(image)
    

In [12]:
# Compile the model (make sure to specify the loss and optimizer)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4, weight_decay=1e-5),
    loss=tf.keras.losses.MeanSquaredError()
)


In [13]:
log_dir = "../logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [14]:
# Step 5: Train the Model

with tf.device('/GPU:0'):
    history = model.fit(
        train_dataset,
        epochs=20,
        validation_data=val_dataset,
        callbacks=[tensorboard_callback]
    )

Epoch 1/20
1154/1154 [==============================] - 15s 10ms/step - loss: 0.0347 - val_loss: 0.0139
Epoch 2/20
1154/1154 [==============================] - 11s 9ms/step - loss: 0.0122 - val_loss: 0.0107
Epoch 3/20
1154/1154 [==============================] - 11s 9ms/step - loss: 0.0107 - val_loss: 0.0093
Epoch 4/20
1154/1154 [==============================] - 11s 9ms/step - loss: 0.0094 - val_loss: 0.0098
Epoch 5/20
1154/1154 [==============================] - 11s 10ms/step - loss: 0.0084 - val_loss: 0.0080
Epoch 6/20
1154/1154 [==============================] - 11s 10ms/step - loss: 0.0075 - val_loss: 0.0076
Epoch 7/20
1154/1154 [==============================] - 11s 10ms/step - loss: 0.0068 - val_loss: 0.0072
Epoch 8/20
1154/1154 [==============================] - 11s 10ms/step - loss: 0.0061 - val_loss: 0.0072
Epoch 9/20
1154/1154 [==============================] - 11s 10ms/step - loss: 0.0054 - val_loss: 0.0077
Epoch 10/20
1154/1154 [==============================] - 11s 10ms/s

In [15]:
# Step 6: Evaluate the Model
model.evaluate(test_dataset)

384/384 [==============================] - 3s 3ms/step - loss: 0.0103


0.010286141186952591